In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 데이터셋 다운로드 

In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="krcQlWTnG5PUWdI1yzWt")
project = rf.workspace("license-plate-mhig5").project("license-plate-7egee")
dataset = project.version(5).download("yolov5")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 8.3 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=87deef5aa21f12f254093f151a6d2580c0dc292ee7ba5dc49f19718eea2c9aa7
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pypars

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to license-plate-5 in yolov5pytorch:: 100%|██████████| 3034/3034 [00:01<00:00, 2680.29it/s]


# YOL0_v5 github clone

In [3]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 15002, done.
remote: Total 15002 (delta 0), reused 0 (delta 0), pack-reused 15002
Receiving objects: 100% (15002/15002), 14.08 MiB | 10.06 MiB/s, done.
Resolving deltas: 100% (10291/10291), done.


In [4]:
%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.5 MB/s eta 0:00:00


# train, val, test 데이터 경로 설정

In [5]:
%cd /
from glob import glob
data_dir = '/content/license-plate-5'
test_list = glob(data_dir+'/test/images/*.jpg')
train_list = glob(data_dir+'/train/images/*.jpg')
val_list = glob(data_dir+'/valid/images/*.jpg')
print(len(train_list), len(val_list), len(test_list))

/
1050 309 152


# train, val, test txt파일 생성

In [6]:
with open(data_dir+'/train.txt', 'w') as f:
  f.write('\n'.join(train_list)+'\n')

with open(data_dir+'/val.txt', 'w') as f:
  f.write('\n'.join(val_list)+'\n')

with open(data_dir+'/test.txt', 'w') as f:
  f.write('\n'.join(test_list)+'\n')

# data.yaml 파일 설정

In [7]:
import yaml

with open(data_dir+'/data.yaml', 'r') as f:
  data = yaml.safe_load(f)
print(data)

data['train'] = data_dir+'/train.txt'
data['val'] = data_dir+'/val.txt'
data['test'] = data_dir+'/test.txt'

with open(data_dir+'/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'names': ['LP'], 'nc': 1, 'roboflow': {'license': 'CC BY 4.0', 'project': 'license-plate-7egee', 'url': 'https://universe.roboflow.com/project/license-plate-7egee/dataset/5', 'version': 5, 'workspace': 'project'}, 'test': '../test/images', 'train': 'license-plate-5/train/images', 'val': 'license-plate-5/valid/images'}
{'names': ['LP'], 'nc': 1, 'roboflow': {'license': 'CC BY 4.0', 'project': 'license-plate-7egee', 'url': 'https://universe.roboflow.com/project/license-plate-7egee/dataset/5', 'version': 5, 'workspace': 'project'}, 'test': '/content/license-plate-5/test.txt', 'train': '/content/license-plate-5/train.txt', 'val': '/content/license-plate-5/val.txt'}


# train

In [8]:
%cd /content/yolov5/

!python train.py --img 416 --batch 4 --epoch 10 --data /content/license-plate-5/data.yaml --cfg ./models/yolov5n.yaml --weights yolov5n.pt --name CarNum_yolov5n_results

/content/yolov5
train: weights=yolov5n.pt, cfg=./models/yolov5n.yaml, data=/content/license-plate-5/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=4, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=CarNum_yolov5n_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-72-g064365d Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.

# 동영상 학습

In [9]:
!python detect.py --weights /content/yolov5/runs/train/CarNum_yolov5n_results/weights/best.pt --img 416 --conf 0.4 --source /content/2.mp4

detect: weights=['/content/yolov5/runs/train/CarNum_yolov5n_results/weights/best.pt'], source=/content/2.mp4, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-72-g064365d Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5n summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
video 1/1 (1/158) /content/2.mp4: 256x416 (no detections), 10.6ms
video 1/1 (2/158) /content/2.mp4: 256x416 (no detections), 9.2ms
video 1/1 (3/158) /content/2.mp4: 256x416 (no detections), 9.6ms
video 1/1 (4/158) /content/2.mp4: 256x416 (no detections), 8.5ms
video 1/1 (5/158) /content/2.mp4: 256x416 (no detections